In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# Display setting for dataframe in notebook
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
decimal = 5
pd.set_option('display.float_format', lambda x: f'%.{decimal}f' % x)

# Import Data

In [3]:
#Only run in the first time to import dataset form excel file
# data = pd.read_excel("./data/data_FS.xlsx", sheet_name=['BalanceSheet', 'IncomeStatement', 'Cashflow', 'Sector_ICB'])
# df_BS = data['BalanceSheet']
# df_IS = data['IncomeStatement']
# df_CF = data['Cashflow']
# df_sector = data['Sector_ICB']

# df_BS.to_pickle("./data_pickle/BalanceSheet.pkl")
# df_IS.to_pickle("./data_pickle/IncomeSatement.pkl")
# df_CF.to_pickle("./data_pickle/Cashflow.pkl")
# df_sector.to_pickle("./data_pickle/Sector.pkl")

In [4]:
#From now on, everytime you need to reload the dataset, re-run this cell only
df_BS = pd.read_pickle("./data_pickle/BalanceSheet.pkl")
df_IS = pd.read_pickle("./data_pickle/IncomeSatement.pkl")
df_CF = pd.read_pickle("./data_pickle/Cashflow.pkl")
df_sector = pd.read_pickle("./data_pickle/Sector.pkl")

In [5]:
df_BS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8027 entries, 0 to 8026
Data columns (total 82 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Ticker                            8027 non-null   object 
 1   ComGroupCode                      8027 non-null   object 
 2   en_OrganName                      8027 non-null   object 
 3   YearReport                        8027 non-null   int64  
 4   CurrentAsset                      8027 non-null   int64  
 5   CashAndEquivalent                 8027 non-null   int64  
 6   Cash                              8027 non-null   int64  
 7   CashEquivalent                    8027 non-null   int64  
 8   Investment_ST                     8027 non-null   int64  
 9   TradingSecurities                 8027 non-null   int64  
 10  HTM_ST                            8027 non-null   int64  
 11  Receivable_ST                     8027 non-null   int64  
 12  TradeR

In [6]:
df_IS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8180 entries, 0 to 8179
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ticker                 8180 non-null   object 
 1   ComGroupCode           8180 non-null   object 
 2   en_OrganName           8180 non-null   object 
 3   YearReport             8180 non-null   int64  
 4   RevenueGross           8180 non-null   int64  
 5   RevenueNet             8180 non-null   int64  
 6   CostOfSales            8180 non-null   int64  
 7   GrossProfit            8180 non-null   int64  
 8   SellingExpense         8180 non-null   int64  
 9   AdministrativeExpense  8180 non-null   int64  
 10  SGA                    8180 non-null   int64  
 11  OperatingProfit        8180 non-null   int64  
 12  FinanceIncome          8180 non-null   int64  
 13  FinanceCost            8026 non-null   float64
 14  FinanceInterestCost    7956 non-null   float64
 15  Fina

In [7]:
df_CF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8144 entries, 0 to 8143
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Ticker        8144 non-null   object 
 1   ComGroupCode  8144 non-null   object 
 2   en_OrganName  8144 non-null   object 
 3   YearReport    8144 non-null   int64  
 4   Depreciation  7875 non-null   float64
 5   CFO           8144 non-null   int64  
 6   CFI           8144 non-null   int64  
 7   CFF           8144 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 509.1+ KB


In [8]:
df_FS = pd.merge(df_BS, df_IS, on=['Ticker', 'YearReport', 'ComGroupCode', 'en_OrganName'], how='inner')
df_FS.shape

(8018, 101)

In [9]:
df_FS = pd.merge(df_FS, df_CF, on=['Ticker', 'YearReport', 'ComGroupCode', 'en_OrganName'], how='inner')
df_FS.shape

(7982, 105)

# Data exploration

In [11]:
col_list = ['Ticker', 'YearReport', 'ComGroupCode', 'en_OrganName']
df_FS[col_list].nunique(axis=0)

Ticker          1689
YearReport         5
ComGroupCode       6
en_OrganName    1689
dtype: int64

In [13]:
count_year = (df_FS['YearReport'].value_counts()).rename_axis('YearReport').reset_index(name='count')
count_year

,YearReport,count
0,2018,1617
1,2019,1615
2,2020,1613
3,2021,1604
4,2022,1533


In [14]:
count_groupcode = (df_FS['ComGroupCode'].value_counts()).rename_axis('ComGroupCode').reset_index(name='count')
count_groupcode

,ComGroupCode,count
0,UpcomIndex,4010
1,VNINDEX,1880
2,HNXIndex,1611
3,OTC,462
4,STOP,18
5,OTHER,1


# Data Cleaning

In [15]:
def data_clean(df):
    #Drop redundant columns with all missing values:
    mask_col = (df.isnull()).all() | (df==0).all()
    col = df.columns[mask_col]
    df.drop(columns=col, inplace = True)
    print('Drop columns with all missing values:', df.shape)
    
    #Drop duplicate rows, keeping the first occurrence
    df = df.drop_duplicates(subset=['Ticker', 'YearReport'], keep='first')
    
    return df

In [16]:
df_FS = data_clean(df_FS)

Drop columns with all missing values: (7982, 104)


In [17]:
# def data_filter(df):
#     #Take only cases with Year < 2023
#     df = df[df['YearReport'].astype(int) < 2023]
#     print('Number of records after filtering', df.shape)
    
#     return df

In [19]:
#Drop company not having at least 3 records
def data_restriction(df, count):
    missing_fin = count[count['count'] < 3]
    df = df[~df['Ticker'].isin(missing_fin['Ticker'])]
    print('Drop company not having at least 3 records:', df.shape)
    
    return df

In [20]:
count_fin = (df_FS['Ticker'].value_counts()).rename_axis('Ticker').reset_index(name='count')
count_fin

,Ticker,count
0,MPT,5
1,QCG,5
2,BQB,5
3,PWS,5
4,PXT,5
...,...,...
1684,HSA,1
1685,CDV,1
1686,C12,1
1687,GCF,1


In [21]:
count_fin[count_fin['count'] < 3].shape

(67, 2)

In [22]:
df_FS = data_restriction(df_FS, count_fin)

Drop company not having at least 3 records: (7878, 104)


## Take out only 3 consecutive years of each FS set

In [23]:
#Trace reconcile
def trace_id(df, year, year_lst):
    if year in df['YearReport'].to_list():
        year_lst.append(year)
        if len(year_lst) < df.shape[0]:
            previous_year = year - 1
            if previous_year != -1:
                trace_id(df, previous_year, year_lst)
    return year_lst

In [24]:
#Since we need at least 3 consecutive years for each company
def select_periods(df, fin_id):
    mapping_key = {}
    for i in fin_id:
        mapping_key.update({i: [0]})
        df_tmp = df[df['Ticker']==i]
        df_tmp = df_tmp.sort_values('YearReport', ascending = False)
        for year in df_tmp['YearReport']:
            year_lst = []
            year_lst = trace_id(df_tmp, year, year_lst)
            if len(year_lst) >= 3:
                mapping_key.update({i: year_lst[:3]})
                break
    return mapping_key

In [25]:
fin_id = df_FS['Ticker'].unique().tolist()
mapping_key = select_periods(df_FS, fin_id)
mapping_key

{'VVS': [2022, 2021, 2020],
 'XDC': [2022, 2021, 2020],
 'SBG': [2022, 2021, 2020],
 'HSV': [2022, 2021, 2020],
 'SCV': [2021, 2020, 2019],
 'LYF': [2021, 2020, 2019],
 'CST': [2022, 2021, 2020],
 'BVL': [2022, 2021, 2020],
 'SGI': [2022, 2021, 2020],
 'TOS': [2022, 2021, 2020],
 'VTZ': [2022, 2021, 2020],
 'SSH': [0],
 'BCA': [2022, 2021, 2020],
 'GMH': [2022, 2021, 2020],
 'BIG': [2022, 2021, 2020],
 'CMM': [2022, 2021, 2020],
 'CNA': [2021, 2020, 2019],
 'ODE': [2021, 2020, 2019],
 'HMR': [2022, 2021, 2020],
 'DSD': [2022, 2021, 2020],
 'NO1': [2022, 2021, 2020],
 'MGR': [2022, 2021, 2020],
 'PPT': [2022, 2021, 2020],
 'GPC': [2022, 2021, 2020],
 'LPT': [2022, 2021, 2020],
 'AAA': [2022, 2021, 2020],
 'AAM': [2022, 2021, 2020],
 'AAT': [2022, 2021, 2020],
 'AAV': [2022, 2021, 2020],
 'ABI': [2022, 2021, 2020],
 'ABR': [2022, 2021, 2020],
 'ABS': [2022, 2021, 2020],
 'ABW': [2022, 2021, 2020],
 'ABT': [2022, 2021, 2020],
 'ACBS': [2022, 2021, 2020],
 'ACC': [2022, 2021, 2020],
 'ACE'

In [26]:
df_FS['used_lst'] = df_FS['Ticker'].map(mapping_key)
df_FS['CHECK'] = [a in b for a, b in zip(df_FS['YearReport'], df_FS['used_lst'])]
df_FS.head(10)

,Ticker,ComGroupCode,en_OrganName,YearReport,CurrentAsset,CashAndEquivalent,Cash,CashEquivalent,Investment_ST,TradingSecurities,HTM_ST,Receivable_ST,TradeReceivable_ST,Prepaid_ST,LoanReceivable_ST,OtherReceivable_ST,Inventory,InventoryGross,InventoryProvision,OtherAssets_ST,NonCurrentAsset,Receivable_LT,TradeReceivable_LT,Prepaid_LT,LoanReceivable_LT,OtherReceivable_LT,PPE,TangibleAssets,TangibleAssetsCost,TangibleAssetsDepreciation,FinancialLease,FinancialLeaseCost,FinancialLeaseDepreciation,IntangibleAssets,IntangibleAssetsCost,IntangibleAssetsDepreciation,InvestmentProperties,InvestmentPropertiesCost,InvestmentPropertiesDepreciation,InProgressAsset,Investment_LT,HTM_LT,InvestmentSubsidiaries,InvestmentJoinVenture,InvestmentOther_LT,OtherAssets_LT,TaxDeferAssets,GoodWill,TotalAssets,TotalLiability,Liability_ST,TradePayable_ST,CustomerPrepaid_ST,Debt_ST,OtherLiabilities_ST,Liability_LT,TradePayable_LT,CustomerPrepaid_LT,Debt_LT,TaxDeferLiabilities,OtherLiabilities_LT,TotalEquity,ContributedEquity,EquitySurplus,TreasuryShare,ProfitUndistributed,OtherEquity,MinorityInterest,TotalLiabilityEquity,Investment_SL,HTM_SL,Receivable_SL,TradeReceivable_SL,Prepaid_SL,LoanReceivable_SL,OtherReceivable_SL,OtherAssets_SL,TradePayable_SL,CustomerPrepaid_SL,Debt_SL,OtherLiabilities_SL,RevenueGross,RevenueNet,CostOfSales,GrossProfit,SellingExpense,AdministrativeExpense,SGA,OperatingProfit,FinanceIncome,FinanceCost,FinanceInterestCost,FinanceNetIC,OtherRevenue,OtherCost,OtherNetIC,InvesteeProfit,ProfitBeforeTax,TaxExpense,NetIncome,Depreciation,CFO,CFI,CFF,used_lst,CHECK
2,VVS,UpcomIndex,Vietnam Machine Investment Development Joint S...,2020,963581132037,34273434683,34273434683,0,0,0,0,676801067863,35677063196,3238431195,27617784501,610267788971.00000,249738314644,249738314644,0,2768314847,110299441685,960600000,0,0,0,960600000.00000,23244058658,23244058658,41234202349,-17990143691,0,0,0,16739171661,19585000000,-2845828339,0,0,0,1089500000,62780000000,62780000000,0,0,0.00000,5486111366,0,0,1073880573722,972332880933,944977029333,221488855202,18706181409,255635994847,449145997875.00000,27355851600,27355851600,0,0,0,0.00000,101547692789,100000000000,0,0,1547692789,0.00000,0,1073880573722,62780000000,62780000000,677761667863,35677063196,3238431195,27617784501,611228388971,8254426213,248844706802,18706181409,255635994847,449145997875,1325960919644,1321853041098,-1221313798219,100539242879,-55495756837,-41519399674,-97015156511,3524086368,29229883459,-23126337367.00000,-13811349323.00000,6103546092,15202439,-38208631,-23006192,0,9604626268,-329109988,9275516280,0.00000,51223331791,-44639466049,15937103865,"[2022, 2021, 2020]",True
3,VVS,UpcomIndex,Vietnam Machine Investment Development Joint S...,2021,2483971784971,78198640901,33875457920,44323182981,120119199806,0,120119199806,1481625886045,63757582089,11150961165,0,1406717342791.00000,749607512344,749607512344,0,54420545875,373000184845,1310600000,0,0,0,1310600000.00000,21867022236,21867022236,45028034649,-23161012413,0,0,0,16013494701,19585000000,-3571505299,0,0,0,1374500000,327566915039,327566915039,0,0,0.00000,4867652869,0,0,2856971969816,2632580546536,2553191632536,539862525752,51761474203,293098087573,1668469545008.00000,79388914000,79388914000,0,0,0,0.00000,224391423280,205000000000,250000000,0,19141423280,0.00000,0,2856971969816,447686114845,447686114845,1482936486045,63757582089,11150961165,0,1408027942791,59288198744,619251439752,51761474203,293098087573,1668469545008,3205544511385,3202747152932,-3074809750320,127937402612,-134989754062,-44603619484,-179593373546,-51655970934,118897345952,-45584954174.00000,-18166868814.00000,73312391778,537450075,-200798017,336652058,0,21993072902,-4399342411,17593730491,0.00000,944023360862,-1042810117415,142712092726,"[2022, 2021, 2020]",True
4,VVS,UpcomIndex,Vietnam Machine Investment Development Joint S...,2022,3251152456524,122408265073,87542011706,34866253367,0,0,0,2636377256629,380196036588,14880752498,0,2241300467543.00000,480105430996,480105430

In [27]:
df_FS = df_FS[df_FS['CHECK']].drop(columns = ['used_lst', 'CHECK'])

In [28]:
g = df_FS.groupby(['Ticker', 'ComGroupCode','en_OrganName']).cumcount()
conditions = [g.values == 0, g.values == 1, g.values == 2]
choices = ['_t0', '_t1', '_t2']
g = pd.Series(np.select(conditions, choices), g.index)

In [29]:
df_FS = (df_FS.set_index(['Ticker','ComGroupCode', 'en_OrganName', g]).unstack().reset_index())

In [30]:
df_FS.columns = [''.join(col) for col in df_FS.columns]

In [31]:
print(df_FS.shape)
df_FS.head(10)

(1601, 306)


,Ticker,ComGroupCode,en_OrganName,YearReport_t0,YearReport_t1,YearReport_t2,CurrentAsset_t0,CurrentAsset_t1,CurrentAsset_t2,CashAndEquivalent_t0,CashAndEquivalent_t1,CashAndEquivalent_t2,Cash_t0,Cash_t1,Cash_t2,CashEquivalent_t0,CashEquivalent_t1,CashEquivalent_t2,Investment_ST_t0,Investment_ST_t1,Investment_ST_t2,TradingSecurities_t0,TradingSecurities_t1,TradingSecurities_t2,HTM_ST_t0,HTM_ST_t1,HTM_ST_t2,Receivable_ST_t0,Receivable_ST_t1,Receivable_ST_t2,TradeReceivable_ST_t0,TradeReceivable_ST_t1,TradeReceivable_ST_t2,Prepaid_ST_t0,Prepaid_ST_t1,Prepaid_ST_t2,LoanReceivable_ST_t0,LoanReceivable_ST_t1,LoanReceivable_ST_t2,OtherReceivable_ST_t0,OtherReceivable_ST_t1,OtherReceivable_ST_t2,Inventory_t0,Inventory_t1,Inventory_t2,InventoryGross_t0,InventoryGross_t1,InventoryGross_t2,InventoryProvision_t0,InventoryProvision_t1,InventoryProvision_t2,OtherAssets_ST_t0,OtherAssets_ST_t1,OtherAssets_ST_t2,NonCurrentAsset_t0,NonCurrentAsset_t1,NonCurrentAsset_t2,Receivable_LT_t0,Receivable_LT_t1,Receivable_LT_t2,TradeReceivable_LT_t0,TradeReceivable_LT_t1,TradeReceivable_LT_t2,Prepaid_LT_t0,Prepaid_LT_t1,Prepaid_LT_t2,LoanReceivable_LT_t0,LoanReceivable_LT_t1,LoanReceivable_LT_t2,OtherReceivable_LT_t0,OtherReceivable_LT_t1,OtherReceivable_LT_t2,PPE_t0,PPE_t1,PPE_t2,TangibleAssets_t0,TangibleAssets_t1,TangibleAssets_t2,TangibleAssetsCost_t0,TangibleAssetsCost_t1,TangibleAssetsCost_t2,TangibleAssetsDepreciation_t0,TangibleAssetsDepreciation_t1,TangibleAssetsDepreciation_t2,FinancialLease_t0,FinancialLease_t1,FinancialLease_t2,FinancialLeaseCost_t0,FinancialLeaseCost_t1,FinancialLeaseCost_t2,FinancialLeaseDepreciation_t0,FinancialLeaseDepreciation_t1,FinancialLeaseDepreciation_t2,IntangibleAssets_t0,IntangibleAssets_t1,IntangibleAssets_t2,IntangibleAssetsCost_t0,IntangibleAssetsCost_t1,IntangibleAssetsCost_t2,IntangibleAssetsDepreciation_t0,IntangibleAssetsDepreciation_t1,IntangibleAssetsDepreciation_t2,InvestmentProperties_t0,InvestmentProperties_t1,InvestmentProperties_t2,InvestmentPropertiesCost_t0,InvestmentPropertiesCost_t1,InvestmentPropertiesCost_t2,InvestmentPropertiesDepreciation_t0,InvestmentPropertiesDepreciation_t1,InvestmentPropertiesDepreciation_t2,InProgressAsset_t0,InProgressAsset_t1,InProgressAsset_t2,Investment_LT_t0,Investment_LT_t1,Investment_LT_t2,HTM_LT_t0,HTM_LT_t1,HTM_LT_t2,InvestmentSubsidiaries_t0,InvestmentSubsidiaries_t1,InvestmentSubsidiaries_t2,InvestmentJoinVenture_t0,InvestmentJoinVenture_t1,InvestmentJoinVenture_t2,InvestmentOther_LT_t0,InvestmentOther_LT_t1,InvestmentOther_LT_t2,OtherAssets_LT_t0,OtherAssets_LT_t1,OtherAssets_LT_t2,TaxDeferAssets_t0,TaxDeferAssets_t1,TaxDeferAssets_t2,GoodWill_t0,GoodWill_t1,GoodWill_t2,TotalAssets_t0,TotalAssets_t1,TotalAssets_t2,TotalLiability_t0,TotalLiability_t1,TotalLiability_t2,Liability_ST_t0,Liability_ST_t1,Liability_ST_t2,TradePayable_ST_t0,TradePayable_ST_t1,TradePayable_ST_t2,CustomerPrepaid_ST_t0,CustomerPrepaid_ST_t1,CustomerPrepaid_ST_t2,Debt_ST_t0,Debt_ST_t1,Debt_ST_t2,OtherLiabilities_ST_t0,OtherLiabilities_ST_t1,OtherLiabilities_ST_t2,Liability_LT_t0,Liability_LT_t1,Liability_LT_t2,TradePayable_LT_t0,TradePayable_LT_t1,TradePayable_LT_t2,CustomerPrepaid_LT_t0,CustomerPrepaid_LT_t1,CustomerPrepaid_LT_t2,Debt_LT_t0,Debt_LT_t1,Debt_LT_t2,TaxDeferLiabilities_t0,TaxDeferLiabilities_t1,TaxDeferLiabilities_t2,OtherLiabilities_LT_t0,OtherLiabilities_LT_t1,OtherLiabilities_LT_t2,TotalEquity_t0,TotalEquity_t1,TotalEquity_t2,ContributedEquity_t0,ContributedEquity_t1,ContributedEquity_t2,EquitySurplus_t0,EquitySurplus_t1,EquitySurplus_t2,TreasuryShare_t0,TreasuryShare_t1,TreasuryShare_t2,ProfitUndistributed_t0,ProfitUndistributed_t1,ProfitUndistributed_t2,OtherEquity_t0,OtherEquity_t1,OtherEquity_t2,MinorityInterest_t0,MinorityInterest_t1,MinorityInterest_t2,TotalLiabilityEquity_t0,TotalLiabilityEquity_t1,TotalLiabilityEquity_t2,Investment_SL_t0,Investment_SL_t1,Investment_SL_t2,HTM_SL_t0,HTM_SL_t1,HTM_SL_t2,Receivable_SL_t0,Receivable_SL_t1,Receivable_SL_t2,TradeRece

## Mapping firm information 

## Sector 

In [36]:
df_sector.head()

,Ticker,ComGroupCode,en_OrganName,en_OrganshortName,ICB_Code_L4,ICB_Name_L4,ICB_Code_L3,ICB_Name_L3,ICB_Code_L2,ICB_Name_L2,ICB_Code_L1,ICB_Name_L1
0,DDB,VNINDEX,Dong Duong Construction And Trading Joint Stoc...,"Dct ., Jsc",2357,Heavy Construction,2350,Construction & Materials,2300,Construction & Materials,2000,Industrials
1,VVS,UpcomIndex,Vietnam Machine Investment Development Joint S...,Vietnam Machine Investment Development,3353,Automobiles,3350,Automobiles & Parts,3300,Automobiles & Parts,3000,Consumer Goods
2,TAB,UpcomIndex,Freco Viet Nam Corporation,"Freco .,corp",2779,Trucking,2770,Industrial Transportation,2700,Industrial Goods & Services,2000,Industrials
3,VMK,UpcomIndex,Vimarko Joint Stock Company,Vimarko,7535,Conventional Electricity,7530,Electricity,7500,Utilities,7000,Utilities
4,HIO,UpcomIndex,Helio Energy Joint Stock Company,"Helio Energy.,jsc",7535,Conventional Electricity,7530,Electricity,7500,Utilities,7000,Utilities


In [37]:
sector_col = df_sector.columns
sector_col

Index(['Ticker', 'ComGroupCode', 'en_OrganName', 'en_OrganshortName', 'ICB_Code_L4', 'ICB_Name_L4', 'ICB_Code_L3', 'ICB_Name_L3', 'ICB_Code_L2',
       'ICB_Name_L2', 'ICB_Code_L1', 'ICB_Name_L1'],
      dtype='object')

In [38]:
df_FS = pd.merge(df_FS, df_sector, on=['Ticker', 'ComGroupCode', 'en_OrganName'], how='inner')
print(df_FS.shape)
df_FS.head()

(1597, 315)


,Ticker,ComGroupCode,en_OrganName,YearReport_t0,YearReport_t1,YearReport_t2,CurrentAsset_t0,CurrentAsset_t1,CurrentAsset_t2,CashAndEquivalent_t0,CashAndEquivalent_t1,CashAndEquivalent_t2,Cash_t0,Cash_t1,Cash_t2,CashEquivalent_t0,CashEquivalent_t1,CashEquivalent_t2,Investment_ST_t0,Investment_ST_t1,Investment_ST_t2,TradingSecurities_t0,TradingSecurities_t1,TradingSecurities_t2,HTM_ST_t0,HTM_ST_t1,HTM_ST_t2,Receivable_ST_t0,Receivable_ST_t1,Receivable_ST_t2,TradeReceivable_ST_t0,TradeReceivable_ST_t1,TradeReceivable_ST_t2,Prepaid_ST_t0,Prepaid_ST_t1,Prepaid_ST_t2,LoanReceivable_ST_t0,LoanReceivable_ST_t1,LoanReceivable_ST_t2,OtherReceivable_ST_t0,OtherReceivable_ST_t1,OtherReceivable_ST_t2,Inventory_t0,Inventory_t1,Inventory_t2,InventoryGross_t0,InventoryGross_t1,InventoryGross_t2,InventoryProvision_t0,InventoryProvision_t1,InventoryProvision_t2,OtherAssets_ST_t0,OtherAssets_ST_t1,OtherAssets_ST_t2,NonCurrentAsset_t0,NonCurrentAsset_t1,NonCurrentAsset_t2,Receivable_LT_t0,Receivable_LT_t1,Receivable_LT_t2,TradeReceivable_LT_t0,TradeReceivable_LT_t1,TradeReceivable_LT_t2,Prepaid_LT_t0,Prepaid_LT_t1,Prepaid_LT_t2,LoanReceivable_LT_t0,LoanReceivable_LT_t1,LoanReceivable_LT_t2,OtherReceivable_LT_t0,OtherReceivable_LT_t1,OtherReceivable_LT_t2,PPE_t0,PPE_t1,PPE_t2,TangibleAssets_t0,TangibleAssets_t1,TangibleAssets_t2,TangibleAssetsCost_t0,TangibleAssetsCost_t1,TangibleAssetsCost_t2,TangibleAssetsDepreciation_t0,TangibleAssetsDepreciation_t1,TangibleAssetsDepreciation_t2,FinancialLease_t0,FinancialLease_t1,FinancialLease_t2,FinancialLeaseCost_t0,FinancialLeaseCost_t1,FinancialLeaseCost_t2,FinancialLeaseDepreciation_t0,FinancialLeaseDepreciation_t1,FinancialLeaseDepreciation_t2,IntangibleAssets_t0,IntangibleAssets_t1,IntangibleAssets_t2,IntangibleAssetsCost_t0,IntangibleAssetsCost_t1,IntangibleAssetsCost_t2,IntangibleAssetsDepreciation_t0,IntangibleAssetsDepreciation_t1,IntangibleAssetsDepreciation_t2,InvestmentProperties_t0,InvestmentProperties_t1,InvestmentProperties_t2,InvestmentPropertiesCost_t0,InvestmentPropertiesCost_t1,InvestmentPropertiesCost_t2,InvestmentPropertiesDepreciation_t0,InvestmentPropertiesDepreciation_t1,InvestmentPropertiesDepreciation_t2,InProgressAsset_t0,InProgressAsset_t1,InProgressAsset_t2,Investment_LT_t0,Investment_LT_t1,Investment_LT_t2,HTM_LT_t0,HTM_LT_t1,HTM_LT_t2,InvestmentSubsidiaries_t0,InvestmentSubsidiaries_t1,InvestmentSubsidiaries_t2,InvestmentJoinVenture_t0,InvestmentJoinVenture_t1,InvestmentJoinVenture_t2,InvestmentOther_LT_t0,InvestmentOther_LT_t1,InvestmentOther_LT_t2,OtherAssets_LT_t0,OtherAssets_LT_t1,OtherAssets_LT_t2,TaxDeferAssets_t0,TaxDeferAssets_t1,TaxDeferAssets_t2,GoodWill_t0,GoodWill_t1,GoodWill_t2,TotalAssets_t0,TotalAssets_t1,TotalAssets_t2,TotalLiability_t0,TotalLiability_t1,TotalLiability_t2,Liability_ST_t0,Liability_ST_t1,Liability_ST_t2,TradePayable_ST_t0,TradePayable_ST_t1,TradePayable_ST_t2,CustomerPrepaid_ST_t0,CustomerPrepaid_ST_t1,CustomerPrepaid_ST_t2,Debt_ST_t0,Debt_ST_t1,Debt_ST_t2,OtherLiabilities_ST_t0,OtherLiabilities_ST_t1,OtherLiabilities_ST_t2,Liability_LT_t0,Liability_LT_t1,Liability_LT_t2,TradePayable_LT_t0,TradePayable_LT_t1,TradePayable_LT_t2,CustomerPrepaid_LT_t0,CustomerPrepaid_LT_t1,CustomerPrepaid_LT_t2,Debt_LT_t0,Debt_LT_t1,Debt_LT_t2,TaxDeferLiabilities_t0,TaxDeferLiabilities_t1,TaxDeferLiabilities_t2,OtherLiabilities_LT_t0,OtherLiabilities_LT_t1,OtherLiabilities_LT_t2,TotalEquity_t0,TotalEquity_t1,TotalEquity_t2,ContributedEquity_t0,ContributedEquity_t1,ContributedEquity_t2,EquitySurplus_t0,EquitySurplus_t1,EquitySurplus_t2,TreasuryShare_t0,TreasuryShare_t1,TreasuryShare_t2,ProfitUndistributed_t0,ProfitUndistributed_t1,ProfitUndistributed_t2,OtherEquity_t0,OtherEquity_t1,OtherEquity_t2,MinorityInterest_t0,MinorityInterest_t1,MinorityInterest_t2,TotalLiabilityEquity_t0,TotalLiabilityEquity_t1,TotalLiabilityEquity_t2,Investment_SL_t0,Investment_SL_t1,Investment_SL_t2,HTM_SL_t0,HTM_SL_t1,HTM_SL_t2,Receivable_SL_t0,Receivable_SL_t1,Receivable_SL_t2,TradeRece

In [39]:
df_FS[sector_col].nunique()

Ticker               1597
ComGroupCode            5
en_OrganName         1597
en_OrganshortName    1596
ICB_Code_L4            84
ICB_Name_L4            84
ICB_Code_L3            35
ICB_Name_L3            35
ICB_Code_L2            18
ICB_Name_L2            18
ICB_Code_L1            10
ICB_Name_L1            10
dtype: int64

In [47]:
count_sector = (df_FS[['ICB_Name_L1', 'ICB_Name_L2']].value_counts()).rename_axis(['ICB_Name_L1', 'ICB_Name_L2']).reset_index(name='count')
count_sector

,ICB_Name_L1,ICB_Name_L2,count
0,Industrials,Construction & Materials,328
1,Industrials,Industrial Goods & Services,266
2,Consumer Goods,Food & Beverage,151
3,Utilities,Utilities,149
4,Financials,Real Estate,122
5,Basic Materials,Basic Resources,115
6,Consumer Goods,Personal & Household Goods,81
7,Basic Materials,Chemicals,70
8,Health Care,Health Care,61
9,Consumer Services,Travel & Leisure,52


## Firm Size 

# Save data

In [28]:
# df_FS.to_pickle("./data_pickle/data_FS.pkl")

In [48]:
df_FS.to_pickle("./data_pickle/data_FS_v1.pkl")